In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from operator import add
import seaborn as sns

In [2]:
def data_split_size(data,size):
      #on a given dataset return the splitted data=> train_data(based on size),validate_data(leftover)
      a_list=np.arange(data.shape[0])
      np.random.shuffle(a_list)
      alist=a_list[0:size]
      train_data=data[alist]
      d=np.arange(data.shape[0])
      leftover=np.delete(d,alist)
      validate_data=data[leftover]
      return train_data,validate_data
    
    
def label_data(data,stest_pred):
      # create label for data(if predicted vale is >/< 5% of error then it labels it '1' or else it is '0')
      ones=np.ones(stest_pred.shape[0])
      zeros= np.zeros(stest_pred.shape[0])
      #print('test shape:',test_data[:,-1].shape,'zeros shape:',zeros.shape,'ones shape:',ones.shape,'stest shape',stest_pred.flatten().shape)
      result = np.where(np.absolute((data[:,-1]-stest_pred.flatten())) > (0.05*np.absolute(data[:,-1])),ones,zeros)
      data[:,-1]=result
      return data
    
def accuracy_estim(labeled_data):
    index_f = np.where(labeled_data[:,-1]==1)
    index_p = np.where(labeled_data[:,-1]==0)
    failed_gt= labeled_data[index_f[0]]
    passed_gt=labeled_data[index_p[0]]
    return (passed_gt.shape[0]/(passed_gt.shape[0]+failed_gt.shape[0]))

In [3]:
train_data= np.loadtxt("./dataware/train_data.txt", delimiter=" ",skiprows=0, dtype=np.float32)
test_data= np.loadtxt("./dataware/test_data.txt", delimiter=" ",skiprows=0, dtype=np.float32)
print(train_data.shape,test_data.shape)

(8000, 5) (3131, 5)


# Random forest

In [4]:
from sklearn.ensemble import RandomForestRegressor
train_X, train_y = train_data[:,:-1],train_data[:,-1]
test_X, test_y = test_data[:,:-1],test_data[:,-1]
print(train_X, train_y)
regr = RandomForestRegressor(random_state=0)

regr.fit(train_X, train_y)
pred_y=regr.predict(test_X)
print('pred_y :',pred_y.shape,'test_y:',test_y.shape)
copied_test_data=np.copy(test_data)
labeled_data= label_data(copied_test_data,pred_y)
print('accuracy estimation:',accuracy_estim(labeled_data))

[[ 0.26        0.69689757  2.         51.32946   ]
 [ 0.5         1.1038107   1.6        15.0969    ]
 [ 0.08        0.46149614  1.6        48.31008   ]
 ...
 [ 0.42        0.75116843  3.6         3.01938   ]
 [ 0.39        0.8934847   1.2         9.05814   ]
 [ 0.41        0.8388722   1.2        21.13566   ]] [150.15158   36.279015 286.4871   ...   6.592417  21.815022  43.81301 ]
pred_y : (3131,) test_y: (3131,)
accuracy estimation: 0.5151708719259023


In [33]:
from sklearn.ensemble import RandomForestRegressor
train_X, train_y = train_data[:,:-1],train_data[:,-1]
test_X, test_y = test_data[:,:-1],test_data[:,-1]
print(train_X, train_y)
regr = RandomForestRegressor(n_estimators=60,max_depth=60)

regr.fit(train_X, train_y)
pred_y=regr.predict(test_X)
print('pred_y :',pred_y.shape,'test_y:',test_y.shape)
copied_test_data=np.copy(test_data)
labeled_data= label_data(copied_test_data,pred_y)
print('accuracy estimation:',accuracy_estim(labeled_data))

[[ 0.26        0.69689757  2.         51.32946   ]
 [ 0.5         1.1038107   1.6        15.0969    ]
 [ 0.08        0.46149614  1.6        48.31008   ]
 ...
 [ 0.42        0.75116843  3.6         3.01938   ]
 [ 0.39        0.8934847   1.2         9.05814   ]
 [ 0.41        0.8388722   1.2        21.13566   ]] [150.15158   36.279015 286.4871   ...   6.592417  21.815022  43.81301 ]
pred_y : (3131,) test_y: (3131,)
accuracy estimation: 0.5212392206962632


# XGBoost

In [5]:
from sklearn import ensemble
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error

regr = ensemble.GradientBoostingRegressor()

regr.fit(train_X, train_y)
pred_y=regr.predict(test_X)
print('pred_y :',pred_y,'test_y:',test_y)
copied_test_data=np.copy(test_data)
labeled_data= label_data(copied_test_data,pred_y)
print('accuracy estimation:',accuracy_estim(labeled_data))

pred_y : [506.19382557 280.90176384 320.18971819 ... 350.46454129 916.34952209
  79.07752811] test_y: [188.67332   70.29463   71.55759  ... 212.18944  739.9296   108.803375]
accuracy estimation: 0.2488022995847972


In [41]:
from sklearn import ensemble
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error

regr = ensemble.GradientBoostingRegressor(learning_rate=0.3,n_estimators=50)

regr.fit(train_X, train_y)
pred_y=regr.predict(test_X)
print('pred_y :',pred_y,'test_y:',test_y)
copied_test_data=np.copy(test_data)
labeled_data= label_data(copied_test_data,pred_y)
print('accuracy estimation:',accuracy_estim(labeled_data))

pred_y : [420.73615003 178.63422142 209.39350642 ... 382.05147255 888.22644562
  73.19304919] test_y: [188.67332   70.29463   71.55759  ... 212.18944  739.9296   108.803375]
accuracy estimation: 0.23059725327371447


# GP regressor

In [6]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel

kernel = DotProduct() + WhiteKernel()
gpr = GaussianProcessRegressor(kernel=kernel,random_state=0).fit(train_X, train_y)

pred_y=gpr.predict(test_X, return_std=False)

In [7]:
copied_test_data=np.copy(test_data)
labeled_data= label_data(copied_test_data,pred_y)
print('accuracy estimation:',accuracy_estim(labeled_data))

accuracy estimation: 0.026509102523155543


In [8]:
print('pred_y :',pred_y,'test_y:',test_y)

pred_y : [427.82035309 348.44164534 394.17641655 ... 425.35305756 763.46434122
 -33.2065582 ] test_y: [188.67332   70.29463   71.55759  ... 212.18944  739.9296   108.803375]
